In [ ]:
#RNN IMPLEMENTATION
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Your input text (Shakespearean plays)
input_text = """
Shakespeare's plays are a canon of approximately 39 dramatic works written by the English poet, playwright, and actor William Shakespeare. The exact number of plays as well as their classifications as tragedy, history, comedy, or otherwise is a matter of scholarly debate. Shakespeare's plays are widely regarded as among the greatest in the English language and are continually performed around the world. The plays have been translated into every major living language.

Many of his plays appeared in print as a series of quartos, but approximately half of them remained unpublished until 1623, when the posthumous First Folio was published. The traditional division of his plays into tragedies, comedies, and histories follows the categories used in the First Folio. However, modern criticism has labelled some of these plays "problem plays" that elude easy categorisation, or perhaps purposely break generic conventions, and has introduced the term romances for what scholars believe to be his later comedies.

When Shakespeare first arrived in London in the late 1580s or early 1590s, dramatists writing for London's new commercial playhouses (such as The Curtain) were combining two strands of dramatic tradition into a new and distinctively Elizabethan synthesis. Previously, the most common forms of popular English theatre were the Tudor morality plays. These plays, generally celebrating piety, use personified moral attributes to urge or instruct the protagonist to choose the virtuous life over Evil. The characters and plot situations are largely symbolic rather than realistic. As a child, Shakespeare would likely have seen this type of play (along with, perhaps, mystery plays and miracle plays).[1]

The other strand of dramatic tradition was classical aesthetic theory. This theory was derived ultimately from Aristotle; in Renaissance England, however, the theory was better known through its Roman interpreters and practitioners. At the universities, plays were staged in a more academic form as Roman closet dramas. These plays, usually performed in Latin, adhered to classical ideas of unity and decorum, but they were also more static, valuing lengthy speeches over physical action. Shakespeare would have learned this theory at grammar school, where Plautus and especially Terence were key parts of the curriculum[2] and were taught in editions with lengthy theoretical introductions.[3]

Theatre and stage setup
Archaeological excavations on the foundations of the Rose and the Globe in the late twentieth century[4] suggested that all London English Renaissance theatres were built around similar general plans. Despite individual differences, the public theatres were three stories high and built around an open space at the center. Usually polygonal in plan to give an overall rounded effect, three levels of inward-facing galleries overlooked the open center into which jutted the stage—essentially a platform surrounded on three sides by the audience, only the rear being restricted for the entrances and exits of the actors and seating for the musicians. The upper level behind the stage could be used as a balcony, as in Romeo and Juliet, or as a position for a character to harangue a crowd, as in Julius Caesar.

Usually built of timber, lath and plaster and with thatched roofs, the early theatres were vulnerable to fire, and gradually were replaced (when necessary) with stronger structures. When the Globe burned down in June 1613, it was rebuilt with a tile roof.

A different model was developed with the Blackfriars Theatre, which came into regular use on a long term basis in 1599. The Blackfriars was small in comparison to the earlier theatres, and roofed rather than open to the sky; it resembled a modern theatre in ways that its predecessors did not.
"""

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([input_text])
total_words = len(tokenizer.word_index) + 1

# Convert text to sequences of tokens
input_sequences = []
for line in input_text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

# Define the RNN model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(SimpleRNN(150, return_sequences=True))
model.add(SimpleRNN(150))  # You can add more layers if desired
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(predictors, label, epochs=15, verbose=1)

# Generate text
seed_text = "Shakespeare's plays are"
next_words = 20
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)[0]
    predicted_index = np.argmax(predicted_probs)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    seed_text += " " + output_word

print("Generated Text:")
print(seed_text)


Epoch 1/15
19/19 [==============================] - 5s 120ms/step - loss: 5.7544 - accuracy: 0.0409
Epoch 2/15
19/19 [==============================] - 3s 159ms/step - loss: 5.4329 - accuracy: 0.0716
Epoch 3/15
19/19 [==============================] - 2s 102ms/step - loss: 5.3654 - accuracy: 0.0716
Epoch 4/15
19/19 [==============================] - 2s 105ms/step - loss: 5.2418 - accuracy: 0.0801
Epoch 5/15
19/19 [==============================] - 2s 105ms/step - loss: 5.0725 - accuracy: 0.0903
Epoch 6/15
19/19 [==============================] - 2s 107ms/step - loss: 4.7773 - accuracy: 0.1397
Epoch 7/15
19/19 [==============================] - 2s 113ms/step - loss: 4.4317 - accuracy: 0.1635
Epoch 8/15
19/19 [==============================] - 3s 164ms/step - loss: 4.0526 - accuracy: 0.2470
Epoch 9/15
19/19 [==============================] - 2s 107ms/step - loss: 3.6586 - accuracy: 0.3356
Epoch 10/15
19/19 [==============================] - 2s 106ms/step - loss: 3.2678 - accuracy: 0.4242

ATTENTION MECHANISM

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Data Preprocessing

# New English sentences
english_sentences = ['start good morning end', 'start see you later end', 'start what are you doing end', 'start I am learning end', 'start have a nice day end']
# Corresponding Turkish sentences
turkish_sentences = ['start günaydın end', 'start görüşürüz end', 'start ne yapıyorsun end', 'start öğreniyorum end', 'start iyi günler end']

# Tokenizing English sentences
english_tokenizer = Tokenizer()
english_tokenizer.fit_on_texts(english_sentences)
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)
encoder_input_data = pad_sequences(english_sequences, padding='post')

# Tokenizing Turkish sentences
turkish_tokenizer = Tokenizer()
turkish_tokenizer.fit_on_texts(turkish_sentences)
turkish_sequences = turkish_tokenizer.texts_to_sequences(turkish_sentences)
decoder_input_data = pad_sequences(turkish_sequences, padding='post')

from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.layers import AdditiveAttention as Attention
from tensorflow.keras.models import Model

# Model parameters
embedding_dim = 256
latent_dim = 512
num_encoder_tokens = len(english_tokenizer.word_index) + 1
num_decoder_tokens = len(turkish_tokenizer.word_index) + 1

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Attention Layer
attention = Attention()
attention_output = attention([decoder_outputs, encoder_outputs])

# Concatenating attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_output])

# Dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Preparing the target data for training
from tensorflow.keras.utils import to_categorical
decoder_target_data = to_categorical(pad_sequences(turkish_sequences, padding='post'), num_decoder_tokens)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=50, validation_split=0.2)


Epoch 1/50
1/1 [==============================] - 8s 8s/step - loss: 2.3009 - accuracy: 0.1250 - val_loss: 2.2803 - val_accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 174ms/step - loss: 2.2481 - accuracy: 0.5000 - val_loss: 2.2633 - val_accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 170ms/step - loss: 2.1989 - accuracy: 0.5000 - val_loss: 2.2456 - val_accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 174ms/step - loss: 2.1403 - accuracy: 0.5000 - val_loss: 2.2269 - val_accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 180ms/step - loss: 2.0657 - accuracy: 0.5000 - val_loss: 2.2100 - val_accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 240ms/step - loss: 1.9712 - accuracy: 0.5000 - val_loss: 2.2065 - val_accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 257ms/step - loss: 1.8640 - accuracy: 0.5000 - val_loss: 2.2415 - val_accuracy: 0.5000
Epoch 8/50
1/1 [=======